In [46]:
import pandas as pd
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import train_test_split
import xgboost as xgb
import numpy as np
from sklearn.preprocessing import LabelEncoder
import optuna
from sklearn.model_selection import KFold
import seaborn as sns

def weighted_mean_absolute_error(y_true, y_pred, weights):
    return (weights * np.abs(y_true - y_pred)).mean()


df = pd.read_csv(r"C:\Users\lllku\Downloads\Telegram Desktop\data.csv")
feature_description = pd.read_csv(r"C:\Users\lllku\Downloads\Telegram Desktop\features_description.csv")
feature_digital_pr = feature_description[feature_description['описание'] == 'данные цифрового профиля']
df = df.drop(columns='dt')

C:\Users\lllku\AppData\Local\Temp\ipykernel_15976\3418161724.py:15: DtypeWarning: Columns (66) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\lllku\Downloads\Telegram Desktop\data.csv")


In [47]:
categorical_inf = []
for column in df.columns:
    if len(df[column].unique()) <= 106:
        categorical_inf.append({'column': column, "uniq": df[column].nunique()})
categorical_inf.append({'column': 'dp_ewb_last_employment_position', "uniq": df['dp_ewb_last_employment_position'].nunique()})
categorical_inf.append({'column': 'dp_address_unique_regions', "uniq": df['dp_address_unique_regions'].nunique()})
categorical = pd.DataFrame(categorical_inf)
categorical = categorical.drop(index=0)

In [48]:
for column in categorical['column']:
    if df[column].isna().any():
        df[column] = df[column].fillna(-1)
for cat in categorical['column']:
    df[cat] = df[cat].astype('category')
categorical

,column,uniq
1,adminarea,85
2,age,71
3,incomeValueCategory,12
4,avg_loan_cnt_with_insurance,51
5,city_smart_name,105
6,mob_cnt_days,90
7,pil,12
8,addrref,62
9,bki_total_auto_cnt,15
10,blacklist_flag,2


In [49]:
categorical_over_106 = []
for column in df.select_dtypes('object').columns:
    if df[column].nunique() > 106:
        categorical_over_106.append(column)
for i in categorical_over_106:
    if i[:3] == 'hdb' or i[:3] == 'bki':
        df[i] = pd.to_numeric(df[i], errors='coerce')
df['gender'] = df['gender'].apply(lambda x: 1 if x == 'Мужской' else 0)
df['adminarea'] = df['adminarea'].apply(lambda x: 'nan' if x == -1 else x)
df['city_smart_name'] = df['city_smart_name'].apply(lambda x: 'nan' if x == -1 else x)
df['addrref'] = df['addrref'].apply(lambda x: 'nan' if x == -1 else x)
df['dp_ewb_last_employment_position'] = df['dp_ewb_last_employment_position'].apply(lambda x: 'nan' if x == -1 else x)

In [50]:
for cat in categorical['column']:
    df[cat] = df[cat].astype(str)
    le = LabelEncoder()
    df[cat] = le.fit_transform(df[cat])
df = df.drop(columns=['hdb_bki_total_pil_max_limit', 'hdb_bki_total_cc_max_limit', 'bki_active_auto_cnt', 'hdb_bki_total_micro_cnt', 'hdb_bki_active_pil_max_limit', 'bki_total_ip_max_outstand', 'bki_total_active_products', 'hdb_bki_other_active_ip_outstanding', 'hdb_bki_total_max_limit'])
df

,id,target,turn_cur_cr_avg_act_v2,salary_6to12m_avg,dp_ils_paymentssum_avg_12m,incomeValue,gender,avg_cur_cr_turn,adminarea,turn_cur_cr_avg_v2,...,dp_ils_uniq_companies_1y,avg_6m_travel,avg_6m_government_services,hdb_bki_active_cc_max_overdue,total_rur_amt_cm_avg_period_days_ago_v2,label_Above_1M_share_r1,transaction_category_supermarket_sum_cnt_d15,max_balance_rur_amt_1m_af,w,first_salary_income
0,2,109324.476325,1465144.960,NaN,365346.244634,97366.0,0,124649.0,65,122095.413333,...,2,0.0,57.0,0.00,297.0,0.027027,95,NaN,0.301217,NaN
1,4,25558.028662,303593.660,NaN,NaN,32580.0,0,63653.0,24,75898.415000,...,0,0.0,707.0,67.00,30245.0,NaN,18,NaN,0.695800,NaN
2,5,40666.753098,490754.010,NaN,NaN,96866.0,0,404545.0,36,122688.502500,...,0,422.0,0.0,0.00,210322.0,0.000000,19,NaN,0.515970,NaN
3,6,43856.672058,219875.120,NaN,NaN,43860.0,0,39988.0,75,18322.926667,...,0,0.0,0.0,0.00,7187.0,0.000000,73,NaN,0.478003,NaN
4,7,130420.851992,1750241.845,NaN,NaN,83815.0,0,202439.0,36,291706.974167,...,0,0.0,84.0,0.00,690038.0,0.000000,95,NaN,0.552314,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76781,149993,68976.691400,NaN,NaN,NaN,NaN,0,NaN,0,NaN,...,0,NaN,NaN,NaN,0.0,NaN,0,NaN,0.179016,NaN
76782,149994,50768.829700,NaN,NaN,NaN,NaN,0,NaN,0,NaN,...,0,NaN,NaN,NaN,0.0,NaN,0,NaN,0.395732,NaN
76783,149996,30613.920000,432098.090,32967.213362,NaN,153541.0,0,43428.0,0,36008.174167,...,0,0.0,42.0,4798.00,20802.0,NaN,18,NaN,0.635623,NaN
76784,149998,81547.761138,3485.000,NaN,NaN,118742.0,0,1162.0,83,290.416667,...,0,NaN,NaN,NaN,0.0,0.000000,0,NaN,0.029391,NaN


In [64]:
X = df.drop(columns=['target', 'w'])
y = df[['target', 'w']]

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = xgb.XGBRegressor(max_depth=6, n_estimators=200, device='cuda', tree_method='hist')
model.fit(x_train, y_train['target'])


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device='cuda', early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=200,
             n_jobs=None, num_parallel_tree=None, ...)

In [ ]:
imp_score = model.feature_importances_
feature_names = X.columns


feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': imp_score
}).sort_values('importance', ascending=False)
feature_importance_df

,feature,importance
8,turn_cur_cr_avg_v2,0.168759
2,salary_6to12m_avg,0.051109
1,turn_cur_cr_avg_act_v2,0.044025
11,dp_ils_avg_salary_1y,0.026091
90,avg_fdep_db_turn,0.018135
...,...,...
18,turn_cur_db_avg_v2,0.000000
81,turn_fdep_db_avg_v2,0.000000
130,dp_ils_cnt_changes_1y,0.000000
127,dp_ewb_dismissal_due_contract_violation_by_lb_cnt,0.000000


In [ ]:

def objective(trial):
    params = {
        'objective': trial.suggest_categorical("objective", ['reg:squarederror', 'reg:absoluteerror']),
        'n_estimators': trial.suggest_int('n_estimators', 100, 2500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        "early_stopping_rounds": trial.suggest_int('early_stopping_rounds', 10, 50),
        'enable_categorical': True,
        'random_state': 42,
    }
    
    X_trial = X[feature_importance_df['feature'][:trial.suggest_int('top_n_features', 3, 150)]]
    
    X_train, X_test, y_train, y_test = train_test_split(X_trial, y, test_size=0.2, random_state=42)
    
    model = xgb.XGBRegressor(**params, device='cuda', tree_method='hist')
    model.fit(X_train, y_train["target"], eval_set=[(X_test, y_test["target"])], verbose=False)
    

    predict = model.predict(X_test)
    return weighted_mean_absolute_error(y_test["target"], predict, y_test["w"])


study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200, n_jobs=-1)

# Results
print("Best trial:")
trial = study.best_trial
print(f"MAPE loss: {trial.value:.4f}")


[I 2025-11-30 12:23:15,599] A new study created in memory with name: no-name-24ef6070-f1c3-4a87-8978-489dee8a5da0
[W 2025-11-30 12:23:15,601] Trial 0 failed with parameters: {'objective': 'reg:squarederror', 'n_estimators': 616, 'max_depth': 6, 'learning_rate': 0.24062985851274057, 'subsample': 0.8079710709810721, 'colsample_bytree': 0.6217672626520254, 'reg_alpha': 1.4945192089160317, 'reg_lambda': 1.7651500586773324e-08, 'early_stopping_rounds': 30} because of the following error: UnboundLocalError("cannot access local variable 'X_train' where it is not associated with a value").
Traceback (most recent call last):
  File "c:\Users\lllku\AppData\Local\Programs\Python\Python312\Lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\lllku\AppData\Local\Temp\ipykernel_15976\2017906220.py", line 16, in objective
    X_trial = X_train[feature_importance_df['feature'][:trial.suggest_int('top_

UnboundLocalError: cannot access local variable 'X_train' where it is not associated with a value

Trial 83 finished with value: 28426.461984512833 and parameters: {'objective': 'reg:squarederror', 'n_estimators': 1848, 'max_depth': 10, 'learning_rate': 0.00806762475574465, 'subsample': 0.9908354562103298, 'colsample_bytree': 0.7908909626596169, 'reg_alpha': 2.5739114786207488e-08, 'reg_lambda': 0.7766280043102454, 'early_stopping_rounds': 45, 'top_n_features': 89}.

In [ ]:
best_params = {'objective': 'reg:squarederror', 'n_estimators': 1848, 'max_depth': 10, 'learning_rate': 0.00806762475574465, 'subsample': 0.9908354562103298, 'colsample_bytree': 0.7908909626596169, 'reg_alpha': 2.5739114786207488e-08, 'reg_lambda': 0.7766280043102454, 'early_stopping_rounds': 45}
def wmae(y_true, y_pred, *, sample_weight):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    sample_weight = np.array(sample_weight)
    return (sample_weight * np.abs(y_true - y_pred)).mean()

In [ ]:
model_test = xgb.XGBRegressor(**best_params, device='cuda', tree_method='hist')
X_trial = X[feature_importance_df['feature'][:89]]#115 best_params

kf = KFold(n_splits=8, shuffle=True, random_state=42)
scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_trial)):
    X_tr = X_trial.iloc[train_idx]
    X_val = X_trial.iloc[val_idx]
    y_tr = y['target'][train_idx]
    y_val = y['target'][val_idx]
    w_tr = y['w'][train_idx]
    w_val = y['w'][val_idx]

    model_test = xgb.XGBRegressor(**params, device='cuda', tree_method='hist')
    model_test.fit(X_tr, y_tr, sample_weight=w_tr, eval_set=[(X_val, y_val)], sample_weight_eval_set=[w_val], verbose=False)

    pred = model_test.predict(X_val)
    wmae_score = np.sum(w_val * np.abs(y_val - pred)) / np.sum(w_val)
    scores.append(wmae_score)
        
    print(f"Fold {fold+1}: WMAE = {wmae_score:.6f} (best_iter: {model_test.best_iteration})")

print(f"\nFinal CV Weighted MAE: {np.mean(scores):.6f} ± {np.std(scores):.5f}")
print(f"Best mean score: {np.mean(scores):.6f}")
# X_train, X_test, y_train, y_test = train_test_split(X_trial, y, test_size=0.2, random_state=42)
# model_test.fit(X_train, y_train["target"], eval_set=[(X_test, y_test["target"])], verbose=False)

c:\Users\lllku\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:729: UserWarning: [10:38:35] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)


Fold 1: WMAE = 48342.408944 (best_iter: 646)
Fold 2: WMAE = 49890.272361 (best_iter: 562)
Fold 3: WMAE = 50238.260764 (best_iter: 609)
Fold 4: WMAE = 50335.456219 (best_iter: 530)
Fold 5: WMAE = 49910.460646 (best_iter: 577)
Fold 6: WMAE = 49799.331038 (best_iter: 604)
Fold 7: WMAE = 49174.134612 (best_iter: 596)
Fold 8: WMAE = 50060.424278 (best_iter: 692)

Final CV Weighted MAE: 49718.843608 ± 615.48213
Best mean score: 49718.843608


In [65]:


df_test = pd.read_csv(r"F:\Browser dwnlds\hackathon_income_test.csv", sep=';', decimal=',')
feature_digital_pr = feature_description[feature_description['описание'] == 'данные цифрового профиля']

# info = []
# for column in df.columns:
#     info.append([column, sum(pd.notna(df[column]))/ ])

df_test

C:\Users\lllku\AppData\Local\Temp\ipykernel_15976\660536909.py:1: DtypeWarning: Columns (65) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv(r"F:\Browser dwnlds\hackathon_income_test.csv", sep=';', decimal=',')


,id,dt,turn_cur_cr_avg_act_v2,salary_6to12m_avg,hdb_bki_total_max_limit,dp_ils_paymentssum_avg_12m,hdb_bki_total_cc_max_limit,incomeValue,gender,avg_cur_cr_turn,...,total_sum,dp_ils_uniq_companies_1y,avg_6m_travel,avg_6m_government_services,hdb_bki_active_cc_max_overdue,total_rur_amt_cm_avg_period_days_ago_v2,label_Above_1M_share_r1,transaction_category_supermarket_sum_cnt_d15,max_balance_rur_amt_1m_af,first_salary_income
0,0,2024-08-31,805319.38,NaN,61137.47,NaN,60000.0,159999.0,Женский,69740.0,...,NaN,NaN,0.0,0.0,NaN,80228.0,0.000000,1.0,NaN,NaN
1,1,2024-10-31,306240.00,NaN,949500.0,NaN,230000.0,108834.0,Мужской,63513.0,...,0.00,NaN,0.0,0.0,0.0,24888.0,0.000000,NaN,NaN,NaN
2,3,2024-09-30,164908.73,NaN,178000.0,NaN,178000.0,59203.0,Женский,132.0,...,38630.63,NaN,0.0,0.0,2363.9,223.0,NaN,NaN,NaN,NaN
3,9,2024-10-31,2374846.42,NaN,25500.0,126247.448359,4999.0,180906.0,Женский,290339.0,...,NaN,3.0,800.0,0.0,NaN,25734.0,0.000000,7.0,0.0,NaN
4,11,2024-11-30,735902.71,47828.145621,60000.0,NaN,60000.0,24922.0,Мужской,76924.0,...,0.00,NaN,0.0,0.0,0.0,1214.0,0.000000,15.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73209,149981,2024-09-30,531284.21,NaN,288500.0,NaN,NaN,64721.0,Мужской,279544.0,...,0.00,NaN,0.0,233.0,NaN,94798.0,NaN,4.0,NaN,NaN
73210,149985,2024-09-30,173804.50,NaN,90000.0,NaN,24999.28,38860.0,Женский,31813.0,...,NaN,NaN,0.0,98.0,30232.0,16785.0,0.000000,NaN,NaN,NaN
73211,149989,2024-10-31,1076401.61,NaN,148000.0,NaN,148000.0,97840.0,Мужской,4233.0,...,NaN,NaN,NaN,NaN,1507.42,524.0,NaN,NaN,NaN,NaN
73212,149995,2024-11-30,2729721.41,NaN,659000.0,132028.008254,160000.0,156088.0,Мужской,187073.0,...,500.00,2.0,11394.0,182.0,0.0,18505.0,0.013333,25.0,1028.0,NaN


In [66]:
for column in categorical['column']:
    if df_test[column].isna().any():
        df_test[column] = df_test[column].fillna(-1)
for cat in categorical['column']:
    df_test[cat] = df_test[cat].astype('category')
categorical

,column,uniq
1,adminarea,85
2,age,71
3,incomeValueCategory,12
4,avg_loan_cnt_with_insurance,51
5,city_smart_name,105
6,mob_cnt_days,90
7,pil,12
8,addrref,62
9,bki_total_auto_cnt,15
10,blacklist_flag,2


In [15]:
X = df_test
ids = X['id']
X = X[feature_importance_df['feature'][:89]]
predictions = model_test.predict(X)

In [ ]:
categorical_over_106 = []
for column in df_test.select_dtypes('object').columns:
    if df_test[column].nunique() > 106:
        categorical_over_106.append(column)
for i in categorical_over_106:
    if i[:3] == 'hdb' or i[:3] == 'bki':
        df_test[i] = pd.to_numeric(df_test[i], errors='coerce')
df_test['gender'] = df_test['gender'].apply(lambda x: 1 if x == 'Мужской' else 0)

In [ ]:
df_test = df_test.drop(columns='dt')
df_test['adminarea'] = df_test['adminarea'].apply(lambda x: 'nan' if x == -1 else x)
df_test['city_smart_name'] = df_test['city_smart_name'].apply(lambda x: 'nan' if x == -1 else x)
df_test['addrref'] = df_test['addrref'].apply(lambda x: 'nan' if x == -1 else x)
df_test['dp_ewb_last_employment_position'] = df_test['dp_ewb_last_employment_position'].apply(lambda x: 'nan' if x == -1 else x)
for cat in categorical['column']:
    df_test[cat] = df_test[cat].astype(str)
    le = LabelEncoder()
    df_test[cat] = le.fit_transform(df_test[cat])

In [18]:
submission = pd.DataFrame({'id': ids, 'target': predictions})
submission.to_csv(index=False, path_or_buf=r"C:\Users\lllku\OneDrive\Документы\submissons.csv")


In [16]:
predictions[:5]

array([63434.062, 59554.375, 40637.5  , 93998.35 , 39516.71 ],
      dtype=float32)